In [1]:
import csv
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier 

In [4]:
df_match_allyears = pd.read_csv("./match_data_yearly/all_years.csv", index_col=0)
df_match_allyears["Date"] = pd.to_datetime(df_match_allyears["Date"])

In [5]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances
ID,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199
...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966


In [6]:
df_recent_stats = pd.read_csv("./recent_stats.csv", index_col=0)
df_recent_stats

,HomeShots,HomeShotsOnTarget,HomeShotsFromPK,HomePasses,HomeCrosses,HomeDirectFK,HomeIndirectFK,HomeCK,HomeThrowin,HomeDribbling,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
10011354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10011229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10012205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10011007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10012178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,20.0,6.8,0.0,494.8,17.8,11.8,2.0,6.8,23.4,12.2,...,24.2,1.8,1.8,0.4,0.0,34.4,13.2,113.4,11.58,45.78
20340810,12.2,4.0,0.2,537.2,11.6,14.2,1.0,3.6,21.8,14.2,...,21.8,1.8,3.0,1.0,0.2,50.4,14.2,127.2,11.30,52.58
20343331,13.0,4.4,0.0,598.6,16.2,9.6,3.2,2.6,17.8,11.4,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20342218,9.0,4.4,0.0,379.8,8.4,13.8,1.2,3.6,18.8,11.4,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62


In [9]:
df_rate = pd.read_csv("./rating/all_years.csv", index_col=0, parse_dates=True)
df_rate

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,tokushima-vortis,yokohama-fc,montedio-yamagata
2006-03-04,1500,1500,1500,1500,1500,1350,1350,1500,1350,1500,...,1500,1500,1350,1500,1500,1350,1500,1350,1350,1350
2006-03-05,1500,1500,1500,1500,1500,1350,1350,1500,1350,1500,...,1500,1500,1350,1500,1500,1350,1500,1350,1350,1350
2006-03-06,1508,1500,1508,1508,1492,1350,1350,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1350,1350
2006-03-07,1508,1500,1508,1508,1492,1350,1350,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1350,1350
2006-03-08,1508,1500,1508,1508,1492,1350,1351,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1351,1350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-16,1595,1490,1543,1678,1535,1411,1478,1541,1444,1577,...,1420,1331,1424,1473,1410,1408,1362,1313,1406,1372
2020-12-17,1595,1486,1543,1682,1526,1419,1487,1541,1455,1582,...,1420,1331,1424,1480,1410,1408,1362,1313,1402,1372
2020-12-18,1595,1486,1543,1682,1526,1419,1487,1541,1455,1582,...,1420,1331,1424,1480,1410,1408,1362,1313,1402,1372
2020-12-19,1595,1486,1543,1682,1526,1419,1487,1541,1455,1582,...,1420,1331,1424,1480,1410,1408,1362,1313,1402,1372


In [11]:
for index,row in df_match_allyears.iterrows():
    home_rate = df_rate.loc[row["Date"], row["Home"]]
    away_rate = df_rate.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomeRate"] = home_rate
    df_match_allyears.at[index,"AwayRate"] = away_rate 

In [12]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate
ID,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1595.0,1559.0
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1514.0,1526.0
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1455.0,1480.0


In [17]:
df_allyears = pd.concat([df_match_allyears,df_recent_stats],axis=1)
df_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
6010524,2006,1,2006-03-05,yokohama-fa-marinos,kyoto-sanga,5,24,4,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011001,2006,1,2006-03-05,sanfrecce-hiroshima,kashima-antlers,10,1,3,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,19.0,2.4,2.2,0.0,0.2,64.0,24.4,115.2,16.08,54.30
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,23.2,2.0,2.4,2.0,0.2,61.2,19.6,124.4,11.58,54.54


In [21]:
df = df_allyears[df_allyears["Year"] >= 2012]
df

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
12010807,2012,1,2012-03-10,nagoya-grampus-eight,shimizu-s-pulse,8,7,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12010918,2012,1,2012-03-10,gamba-osaka,vissel-kobe,9,18,2,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011003,2012,1,2012-03-10,sanfrecce-hiroshima,urawa-red-diamonds,10,3,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011105,2012,1,2012-03-11,kashiwa-reysol,yokohama-fa-marinos,11,5,3,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011413,2012,1,2012-03-10,consadole-sapporo,jubilo-iwata,14,13,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,19.0,2.4,2.2,0.0,0.2,64.0,24.4,115.2,16.08,54.30
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,23.2,2.0,2.4,2.0,0.2,61.2,19.6,124.4,11.58,54.54


### statsの欠損値を前の年の降格チームの平均値で埋める

In [39]:
df_clubs = pd.read_csv("./clubs.csv", index_col=0)
df_clubs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2006,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,sanfrecce-hiroshima,jubilo-iwata,cerezo-osaka,kawasaki-frontale,fc-tokyo,avispa-fukuoka,kyoto-sanga,omiya-ardija,ventforet-kofu,oita-trinita,albirex-niigata
2007,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,sanfrecce-hiroshima,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,ventforet-kofu,oita-trinita,albirex-niigata,vissel-kobe,kashiwa-reysol,yokohama-fc
2008,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,oita-trinita,albirex-niigata,vissel-kobe,kashiwa-reysol,consadole-sapporo,tokyo-verdy,kyoto-sanga
2009,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,oita-trinita,albirex-niigata,vissel-kobe,kashiwa-reysol,kyoto-sanga,sanfrecce-hiroshima,montedio-yamagata
2010,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,albirex-niigata,vissel-kobe,kyoto-sanga,sanfrecce-hiroshima,montedio-yamagata,vegalta-sendai,cerezo-osaka,shonan-bellmare
2011,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,omiya-ardija,albirex-niigata,vissel-kobe,sanfrecce-hiroshima,montedio-yamagata,vegalta-sendai,cerezo-osaka,kashiwa-reysol,ventforet-kofu,avispa-fukuoka
2012,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,omiya-ardija,albirex-niigata,vissel-kobe,sanfrecce-hiroshima,vegalta-sendai,cerezo-osaka,kashiwa-reysol,fc-tokyo,sagan-tosu,consadole-sapporo
2013,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,jubilo-iwata,kawasaki-frontale,omiya-ardija,albirex-niigata,sanfrecce-hiroshima,vegalta-sendai,cerezo-osaka,kashiwa-reysol,fc-tokyo,sagan-tosu,ventforet-kofu,shonan-bellmare,oita-trinita
2014,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,kawasaki-frontale,omiya-ardija,albirex-niigata,sanfrecce-hiroshima,vegalta-sendai,cerezo-osaka,kashiwa-reysol,fc-tokyo,sagan-tosu,ventforet-kofu,gamba-osaka,vissel-kobe,tokushima-vortis
2015,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,kawasaki-frontale,albirex-niigata,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,ventforet-kofu,gamba-osaka,vissel-kobe,shonan-bellmare,matsumoto-yamaga,montedio-yamagata


In [47]:
relegation_clubs = list(set(df_clubs.loc[2019])-set(df_clubs.loc[2020]))
relegation_clubs

['jubilo-iwata', 'matsumoto-yamaga']

In [53]:
recent_stats_items = [ 'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK',
             'HomePasses', 'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
             'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
             'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
             'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
              'HomePossession', 'AwayShots',
             'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
             'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
             'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
             'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
             'AwayPenaltyAreaEntries', 'AwayAttacks', 'AwayPossession'] 

In [54]:
df[
    (df["Year"] == 2015) &
    (
        ((df["Home"] == "shimizu-s-pulse")|(df["Away"] == "shimizu-s-pulse"))|
        ((df["Home"] == "montedio-yamagata")|(df["Away"] == "montedio-yamagata"))|
        ((df["Home"] == "matsumoto-yamaga")|(df["Away"] == "matsumoto-yamaga"))
    )
][recent_stats_items].mean()

HomeShots                  12.945833
HomeShotsOnTarget           4.013889
HomeShotsFromPK             0.092708
HomePasses                415.336458
HomeCrosses                14.883160
HomeDirectFK               11.923090
HomeIndirectFK              2.301389
HomeCK                      4.648611
HomeThrowin                23.505208
HomeDribbling              10.911111
HomeTackles                21.578993
HomeClearances             25.490799
HomeIntercepts              2.920139
HomeOffsides                2.160590
HomeYellowCards             1.295486
HomeRedCards                0.043750
Home30mLineEntries         39.489583
HomePenaltyAreaEntries     12.767708
HomeAttacks               128.632292
HomePossession             47.570538
AwayShots                  13.206206
AwayShotsOnTarget           4.069326
AwayShotsFromPK             0.132270
AwayPasses                417.980496
AwayCrosses                15.280496
AwayDirectFK               12.080496
AwayIndirectFK              2.564539
A

In [57]:
for relegation_club in  relegation_clubs:
     print(df[(df["Year"] == 2019) &((df["Home"] == relegation_club)|(df["Away"] == relegation_club))][recent_stats_items].mean())      
        

HomeShots                  12.882353
HomeShotsOnTarget           3.829412
HomeShotsFromPK             0.082353
HomePasses                471.705882
HomeCrosses                15.135294
HomeDirectFK               12.294118
HomeIndirectFK              2.352941
HomeCK                      4.717647
HomeThrowin                20.935294
HomeDribbling              12.329412
HomeTackles                19.264706
HomeClearances             22.364706
HomeIntercepts              2.529412
HomeOffsides                1.758824
HomeYellowCards             1.123529
HomeRedCards                0.064706
Home30mLineEntries         41.047059
HomePenaltyAreaEntries     12.117647
HomeAttacks               115.582353
HomePossession             49.372353
AwayShots                  13.396970
AwayShotsOnTarget           4.018182
AwayShotsFromPK             0.078788
AwayPasses                477.836364
AwayCrosses                15.230303
AwayDirectFK               12.021212
AwayIndirectFK              2.233333
A